# Code Review Tool: Approach

This AI tool helps developers get feedback on their code - like having a friendly senior developer look over your work and give you tips. It's built with LangGraph, which lets us create a step-by-step workflow where each step handles one part of the review process.

## How It Works

1. **Getting the Code**: The tool asks you to paste in your code and tell it what programming language you're using.

2. **Looking at Structure**: First, it analyzes how your code is organized - are functions well-structured? Is the code clean? 

3. **Finding Issues**: Next, it looks for potential bugs, performance problems, or places where the code could be improved.

4. **Suggesting Improvements**: Based on the issues found, it comes up with specific ways to make your code better.

5. **Writing the Review**: Finally, it puts everything together into a helpful review, giving your code a score and explaining what's good and what could be better.

6. **Next Steps**: The tool asks if you want to review another piece of code. If yes, the process starts over.


## Behind the Scenes

The tool uses AI language models to understand and analyze code. We're using LangSmith to track how it's performing and find ways to make it better. Everything is hooked together in a workflow that passes information from one step to the next.

Think of it like an assembly line for code reviews - each station adds something valuable before passing it to the next one!
![Gemini_Generated_Image_izpoluizpoluizpo](https://github.com/user-attachments/assets/a83a4d01-6045-4124-93d5-093bb53dc44d)


### **🚀 Steps for Implementing the Code Peer Review Workflow**


 **Step 1: Set Up the Environment**
   - Install required libraries: `langchain`, `langgraph`, `transformers`, `huggingface_hub`, `langsmith`.
   - Configure LangSmith for debugging and monitoring.
  
  **!pip install -r requirements.txt**

In [1]:
import langgraph
from langchain.chat_models import ChatOpenAI
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFaceHub
from langsmith import trace
from dotenv import load_dotenv
from langgraph.graph import START, END, StateGraph
from typing import TypedDict, List, Optional
from pydantic import BaseModel
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt import tools_condition
from langchain_core.tracers import LangChainTracer
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.tracers.langchain import wait_for_all_tracers
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")

**Step2: Integrate Open-Source LLM**
   - Use **Groq API** (e.g., `gpt-4o`).
 

In [2]:
#Initialize the open-source LLM (Hugging face)
llm=ChatOpenAI(model="gpt-4o")

/var/folders/qs/0l_hn7gs56g75pysv29fs3dh0000gn/T/ipykernel_5876/635163774.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(model="gpt-4o")


In [3]:
# Initialize tracer
tracer = LangChainTracer(project_name="code-peer-review-project")
callback_manager = CallbackManager([tracer])

class CodeReview(BaseModel):
    code_id: str
    code_content: str
    review_content: str
    issues_found: List[str] = []
    recommendations: List[str] = []
    quality_score: Optional[int] = None

class ReviewState(BaseModel):
    code_submission: Optional[str] = None
    language: Optional[str] = None
    analysis: Optional[str] = None
    issues: List[str] = []
    recommendations: List[str] = []
    review: Optional[str] = None
    reviews: List[CodeReview] = []
    current_review: Optional[CodeReview] = None

class GraphState(TypedDict):
    review:ReviewState

In [7]:
# Define the nodes

def input_code(state: GraphState) -> GraphState:
    """Get input code submission from user"""
    review_state = state["review"]
    review_state.code_submission = input("Paste your code for review: ")
    review_state.language = input("What programming language is this? ")
    return {"review": review_state}

def analyze_code(state: GraphState) -> GraphState:
    """Analyze the submitted code for structure and patterns"""
    review_state = state["review"]
    
    prompt = PromptTemplate(
        input_variables=["code", "language"],
        template="""You are an expert code analyzer. Perform a detailed structural analysis of the following {language} code:
        
Code:
```{language}
{code}
```

Analyze the code for:
1. Overall structure and organization
2. Design patterns used
3. Function and class organization
4. Complexity assessment
5. Any notable coding style patterns

Provide a comprehensive analysis focusing on the architecture and structure.
"""
    )
    
    response = llm.invoke(prompt.format(code=review_state.code_submission, language=review_state.language))
    
    if hasattr(response, 'content'):
        review_state.analysis = response.content
    else:
        review_state.analysis = str(response)
    
    return {"review": review_state}


def identify_issues(state: GraphState) -> GraphState:
    """Identify potential issues, bugs, and code smells in the submitted code"""
    review_state = state["review"]
    
    prompt = PromptTemplate(
        input_variables=["code", "language", "analysis"],
        template="""You are an expert code reviewer specializing in identifying issues and code quality problems. 
Based on the provided code and analysis, identify potential issues, reduce nested conditions like if-else, bugs, code smells, and areas for improvement.

Code:
```{language}
{code}
```

Previous Analysis:
{analysis}

List all issues you can find, categorized by:
1. Critical bugs or security vulnerabilities
2. Performance concerns
3. Code smells and maintainability issues
4. Style and convention violations
5. Potential edge cases not handled

Be specific and provide line references where possible.
"""
    )
    
    response = llm.invoke(prompt.format(
        code=review_state.code_submission, 
        language=review_state.language,
        analysis=review_state.analysis
    ))
    
    if hasattr(response, 'content'):
        issues_content = response.content
    else:
        issues_content = str(response)
    
    # Parse the issues from the response
    # This is a simple approach - in production you might want more structured parsing
    import re
    issues = [line.strip() for line in issues_content.split('\n') if line.strip() and not line.strip().startswith('#')]
    review_state.issues = issues
    
    return {"review": review_state}

def generate_recommendations(state: GraphState) -> GraphState:
    """Generate specific recommendations to improve the code"""
    review_state = state["review"]
    
    prompt = PromptTemplate(
        input_variables=["code", "language", "issues"],
        template="""You are an expert code mentor focused on providing constructive recommendations. 
Based on the submitted code and identified issues, provide specific, actionable recommendations for improvement.

Code:
```{language}
{code}
```

Identified Issues:
{issues}

Provide detailed recommendations that:
1. Address each identified issue with concrete code examples
2. Suggest best practices for this specific context
3. Recommend refactoring approaches where applicable
4. Include explanations of why each recommendation improves the code
5. Consider both immediate fixes and long-term maintainability

Format your recommendations as a numbered list of clear, actionable items with code examples.
"""
    )
    
    issues_text = "\n".join(review_state.issues)
    response = llm.invoke(prompt.format(
        code=review_state.code_submission, 
        language=review_state.language,
        issues=issues_text
    ))
    
    if hasattr(response, 'content'):
        recommendations_content = response.content
    else:
        recommendations_content = str(response)
    
    # Extract recommendations
    import re
    recommendations = [line.strip() for line in recommendations_content.split('\n') 
                      if line.strip() and not line.strip().startswith('#')]
    review_state.recommendations = recommendations
    
    return {"review": review_state}

def compile_review(state: GraphState) -> GraphState:
    """Compile a comprehensive code review from the analysis, issues, and recommendations"""
    review_state = state["review"]
    
    prompt = PromptTemplate(
        input_variables=["code", "language", "analysis", "issues", "recommendations"],
        template="""You are a professional code reviewer creating a comprehensive peer review document. 
Synthesize the following analysis, issues, and recommendations into a well-structured, constructive code review.

Code Language: {language}

Analysis Summary:
{analysis}

Identified Issues:
{issues}

Recommendations:
{recommendations}

Create a professional code review document that:
1. Begins with a brief overview of the code and its purpose
2. Highlights positive aspects and strengths of the code
3. Presents issues in a constructive, non-judgmental manner
4. Provides clear, actionable recommendations with code examples
5. Assigns a quality score from 1-10 with justification
6. Ends with a summary of key takeaways

The tone should be professional, constructive, and educational. Focus on helping the developer improve rather than just pointing out problems.
"""
    )
    
    issues_text = "\n".join(review_state.issues)
    recommendations_text = "\n".join(review_state.recommendations)
    
    response = llm.invoke(prompt.format(
        code=review_state.code_submission, 
        language=review_state.language,
        analysis=review_state.analysis,
        issues=issues_text,
        recommendations=recommendations_text
    ))
    
    if hasattr(response, 'content'):
        review_state.review = response.content
    else:
        review_state.review = str(response)
    
    # Extract quality score - this is a simple regex approach
    import re
    score_match = re.search(r'quality score.*?(\d+)', review_state.review, re.IGNORECASE)
    quality_score = int(score_match.group(1)) if score_match else 5
    
    # Create a new code review with the generated content
    code_id = f"code_review_{len(review_state.reviews) + 1}"
    new_review = CodeReview(
        code_id=code_id,
        code_content=review_state.code_submission,
        review_content=review_state.review,
        issues_found=review_state.issues,
        recommendations=review_state.recommendations,
        quality_score=quality_score
    )
    review_state.reviews.append(new_review)
    review_state.current_review = new_review
    
    return {"review": review_state}

def decide_next(state: GraphState) -> dict:
    """Decision node to determine next steps"""
    print("\nCurrent code review created:")
    print(f"Code ID: {state['review'].current_review.code_id}")
    print(f"Quality Score: {state['review'].current_review.quality_score}/10")
    print(f"Issues Found: {len(state['review'].current_review.issues_found)}")
    print(f"Preview of review:\n{state['review'].current_review.review_content[:200]}...\n")
    
    more_reviews = input("Review another code submission? (y/n): ").lower() == 'y'
    if more_reviews:
        return {"next": "review_more"}
    else:
        return {"next": "end"}

In [8]:
# Create the graph

def build_review_graph():
    # Initialize the state graph
    workflow = StateGraph(GraphState)
    
    # Add the nodes
    workflow.add_node("input_code", input_code)
    workflow.add_node("analyze_code", analyze_code)
    workflow.add_node("identify_issues", identify_issues)
    workflow.add_node("generate_recommendations", generate_recommendations)
    workflow.add_node("compile_review", compile_review)
    workflow.add_node("decide_next", decide_next)
    
    # Add the edges
    workflow.add_edge(START, "input_code")
    workflow.add_edge("input_code", "analyze_code")
    workflow.add_edge("analyze_code", "identify_issues")
    workflow.add_edge("identify_issues", "generate_recommendations")
    workflow.add_edge("generate_recommendations", "compile_review")
    workflow.add_edge("compile_review", "decide_next")
    
    # Define conditional edges using a router
    workflow.add_conditional_edges(
        "decide_next",
        lambda x: x["next"],
        {
            "review_more": "input_code",
            "end": END
        }
    )
    
    # Compile the graph
    graph = workflow.compile()
    return graph

In [9]:
if __name__ == "__main__":
    try:
        # Build the graph
        review_graph = build_review_graph()
        
        # Initialize with the correct state structure
        initial_state = {"review": ReviewState()}
        
        # Run the graph with LangSmith tracing
        final_state = review_graph.invoke(
            initial_state,
            {"callbacks": [tracer]}
        )
        
        # Display results
        print("\nCompleted Code Reviews:")
        for i, review in enumerate(final_state["review"].reviews):
            file_name = f"code_review_{i+1}.md"
            with open(file_name, "w", encoding="utf-8") as f:
                f.write(f"# Code Review: {review.code_id}\n\n")
                f.write(f"## Quality Score: {review.quality_score}/10\n\n")
                f.write(f"## Review\n\n{review.review_content}\n\n")
                f.write("## Original Code\n\n```" + final_state["review"].language + "\n")
                f.write(review.code_content + "\n```\n")
            print(f"Saved complete code review to {file_name}")
        
        # Wait for all traces to be uploaded
        wait_for_all_tracers()
        print("All LangSmith traces uploaded successfully!")
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")


Current code review created:
Code ID: code_review_1
Quality Score: 6/10
Issues Found: 16
Preview of review:
### Code Review Document

#### Overview

The provided C# code implements a method named `CheckSpecsActive` within a class, which is responsible for checking, managing, and potentially caching specific...


Completed Code Reviews:
Saved complete code review to code_review_1.md
All LangSmith traces uploaded successfully!


# Code Review: code_review_1

## Quality Score: 5/10

## Review

# Code Review Document

## Overview

The provided C# code snippet is part of a larger class that includes a private method named `LoadCsvFile`. This method is responsible for determining the appropriate file path for a CSV file based on various conditions and parameter inputs, such as firmware version and New Product Introduction (NPI) phase. It returns a `List<Dictionary<string, string>>`, indicating that its purpose is to produce a structured representation of CSV content.

## Positive Aspects and Strengths


1. **Use of Design Patterns**: The method effectively utilizes the Strategy Pattern to decide between development and production file paths, enhancing flexibility. The use of guard clauses to handle invalid conditions early is a commendable practice that simplifies control flow.

2. **Clear Exception Messages**: The exceptions thrown within the method are informative, providing clear feedback on error conditions which can assist in debugging and understanding issues quickly.

3. **Use of Inline Initialization**: Local variables are initialized inline, which reduces the scope and improves the clarity of the code.

4. **String Manipulation Practices**: The code uses `System.IO.Path.Combine` for file path constructions, ensuring cross-platform compatibility and reducing potential errors in path handling.

## Identified Issues

1. **Generic Exception Usage**: The method throws generic `Exception` objects, which offer limited context for error handling. Using more specific exception types would improve error context.

2. **Redundant File Existence Checks**: The code checks for file existence multiple times, which can be optimized to reduce I/O operations.

3. **Misleading Method Name**: The method name, `LoadCsvFile`, does not align with its primary function of path determination and validation.

4. **Implicit Logic for Environment Determination**: The `dev` variable determination is implicit and could benefit from clearer logic separation.

5. **Magic Numbers**: The use of numbers like `3` and `2` without context decreases code readability and maintainability.

6. **Single Responsibility Violation**: The method handles both path determination and validation, suggesting a need for refactoring to separate concerns.

7. **Readability Issues**: The code lacks sufficient whitespace and uses inline variable declarations, making it harder to read.

8. **Inconsistent Path Construction**: Inconsistent use of path construction methods can lead to potential errors and reduced clarity.

9. **Potential File Path Issues**: Assumptions about path separators at the end of `prodPath` and `devPath` can lead to incorrect path formation.

10. **Unvalidated Input Parameters**: The lack of input validation may lead to unexpected behavior when invalid data is provided.

## Recommendations

1. **Use Specific Exception Types**:
   ```csharp
   throw new InvalidOperationException($"You cannot use dev map files at NPI phase {npiPhase}");
   ```

2. **Optimize File Existence Checks**:
   ```csharp
   bool fwMapTxtExists = File.Exists(fwMapPath + ".txt");
   bool fwMapMapExists = File.Exists(fwMapPath + ".map");
   ```

3. **Rename the Method**:
   ```csharp
   private List<Dictionary<string, string>> DetermineFilePathBasedOnConditions(...) { ... }
   ```

4. **Extract Logic into Separate Methods**:
   ```csharp
   private bool IsDevelopmentEnvironment(string fwVersion, string fwMapPath) { ... }
   ```

5. **Use Named Constants**:
   ```csharp
   private const int MinTestPhaseForDev = 3;
   ```

6. **Separate Concerns**:
   ```csharp
   private string GetFirmwareFilePath(string basePath, string fwVersion) { ... }
   ```

7. **Improve Readability**:
   ```csharp
   var result = new List<Dictionary<string, string>>();
   ```

8. **Consistent Use of Path.Combine**:
   ```csharp
   string fwMapPath = System.IO.Path.Combine(prodPath, fwVersion);
   ```

9. **Handle File Path Construction Robustly**:
   ```csharp
   private string EnsureTrailingSeparator(string path) { ... }
   ```

10. **Validate Input Parameters**:
    ```csharp
    private void ValidateInputs(string fwVersion, string prodPath, string devPath) { ... }
    ```

## Quality Score

**Score: 7/10**

The code demonstrates a good understanding of design patterns and includes useful practices like guard clauses and informative exception messages. However, it suffers from issues related to method naming, redundant operations, single responsibility violations, and lack of input validation. By addressing these, the code can significantly improve in maintainability and clarity.

## Summary of Key Takeaways

- **Design Patterns**: Effective use of strategy and guard clauses enhances flexibility and readability.
- **Exception Handling**: Specific exception types provide better context for error handling.
- **Method Naming**: Accurate naming reflects functionality and improves code comprehension.
- **Code Refactoring**: Breaking down complex methods can enforce single responsibility and improve readability.
- **Input Validation**: Ensures robustness and prevent unexpected errors.

Implementing these recommendations will lead to a more maintainable, robust, and readable codebase, fostering better collaboration and future development efforts.

## Original Code

```C#
private List<Dictionary<string, string>> LoadCsvFile(string fwVersion, int npiPhase, string prodPath, string devPath, List<string> header) {     var result = new List<Dictionary<string, string>>();     LoadedFiles = new List<string>();     string fwMapPath = prodPath + fwVersion;         var dev = fwVersion.ToUpper().Contains("DEV") || (!File.Exists(fwMapPath + ".txt") && !File.Exists(fwMapPath + ".map"));     if (_specsManager.IsSpecValid("TEST_PHASE") && _specsManager.IsSpecValid("FW_DEV"))     {         int testPhase = Convert.ToInt32(_specsManager.GetSpecValue("TEST_PHASE"));         int fwDev = Convert.ToInt32(_specsManager.GetSpecValue("FW_DEV"));          dev = (testPhase < 3 && fwDev == 1);     }      if (dev && npiPhase > 2) throw new Exception($"You cannot use dev map files at NPI phase {npiPhase}");     var basePath = dev ? devPath : prodPath;     var mapName = System.IO.Path.Combine(basePath, $"{fwVersion}.map");     var txtName = System.IO.Path.Combine(basePath, $"{fwVersion}.txt");     var txtFileExists = File.Exists(txtName);     var mapFileExists = File.Exists(mapName);     if (!txtFileExists && !mapFileExists) throw new Exception($"Firmware file {fwVersion} doesn't exist");     else if (mapFileExists && txtFileExists) throw new Exception($"Conflicting files exist (.txt and .map cannot both exist for the same FW version):\n{mapName}\n{txtName}");     fwMapPath = mapFileExists ? mapName : txtName;
```


# Code Review: code_review_2

## Quality Score: 6/10

## Review

### Code Review Document

#### Overview

The provided C# code implements a method named `CheckSpecsActive` within a class, which is responsible for checking, managing, and potentially caching specifications. The method handles specification overrides and returns a dictionary comprising specification data. It is integral to the system's performance optimization strategy, leveraging caching mechanisms to avoid redundant operations.

#### Positive Aspects and Strengths

1. **Cohesive Responsibility**: The method maintains a single responsibility, focusing on managing specifications with caching and override capabilities, which aligns well with the principles of clean code.

2. **Readable Structure**: The method demonstrates a clear structure, supported by meaningful variable names, which enhances its readability and maintainability.

3. **Caching Mechanism**: The use of a caching mechanism (`SpecCache`) indicates an understanding of performance optimization, reducing unnecessary operations and improving efficiency.

4. **Immutable Return**: Returning a new dictionary ensures immutability, minimizing side effects and promoting safe code practices.

#### Issues and Areas for Improvement

1. **Undefined Variable**: The variable `path` is used but not defined, leading to a compilation error. This should be addressed to ensure code completeness.
   
2. **Potential Null References**: The method does not handle potential null references for the `OverrideDict`, which could lead to runtime exceptions.
   
3. **Redundant Cache Set**: The cache operation uses `CacheSpecs` without proper initialization, potentially leading to incorrect caching and performance issues.

4. **Complex Conditional Logic**: The nested conditional logic can be simplified to improve readability and maintainability.

5. **Inconsistent Naming Conventions**: The code uses mixed naming conventions, which affects readability and can lead to confusion.

6. **Lack of Error Handling**: The absence of try-catch blocks around critical operations could lead to unhandled exceptions and application crashes.

7. **Specs Retrieval Failures**: There is no handling for scenarios where `Specs.GetSpecsAsDictionary` might fail or return null, which could result in unexpected behavior.

#### Recommendations

1. **Define and Initialize Variables**:
   - Ensure `path` is defined and initialized before use. Consider passing it as a parameter to make the method self-contained.
   ```csharp
   public Dictionary<string, string> CheckSpecsActive(string testKind, string modelNumber, Dictionary<string, string> overrideDict, string path) { ... }
   ```

2. **Handle Null References**:
   - Add a null check for `OverrideDict` to prevent null reference exceptions.
   ```csharp
   if (overrideDict == null) {
       overrideDict = new Dictionary<string, string>();
   }
   ```

3. **Refactor Cache Handling**:
   - Initialize and correctly handle cache operations to ensure only valid data is cached.
   ```csharp
   if (SpecCache.TryGet(key, out var cacheSpecs)) {
       return cacheSpecs;
   } else {
       Specs.SetActiveSpecs(testKind, modelNumber);
       var activeSpecs = Specs.GetSpecsAsDictionary(testKind, modelNumber);
       SpecCache.Set(key, activeSpecs);
       return activeSpecs;
   }
   ```

4. **Simplify Conditional Logic**:
   - Use early returns to simplify code flow and reduce nesting.
   ```csharp
   if (overrideDict.Count > 0) {
       foreach (var pair in overrideDict) {
           Specs.OverrideSpec(pair.Key, pair.Value);
       }
   }
   return Specs.GetSpecsAsDictionary(testKind, modelNumber);
   ```

5. **Consistent Naming Conventions**:
   - Adopt camelCase for local variables and parameters to enhance consistency and readability.
   ```csharp
   var configObj = new ConfigObject(path);
   var dictSpecs = new Dictionary<string, string>();
   ```

6. **Error Handling with Try-Catch**:
   - Implement try-catch blocks around operations that may fail to handle exceptions gracefully.
   ```csharp
   try {
       Specs.SetActiveSpecs(testKind, modelNumber);
   } catch (Exception ex) {
       // Log exception and handle it
   }
   ```

7. **Handle Specs Retrieval Failures**:
   - Add checks for null or handle exceptions when retrieving specs.
   ```csharp
   var specsDictionary = Specs.GetSpecsAsDictionary(testKind, modelNumber);
   if (specsDictionary == null) {
       // Handle the failure case
   }
   ```

#### Quality Score: 6/10

**Justification**: The code demonstrates a solid understanding of performance optimization through caching and maintains a single responsibility. However, it lacks in areas like error handling, consistent naming, and handling of null references. By addressing these issues, the code's robustness and maintainability can be significantly enhanced.

#### Summary of Key Takeaways

The `CheckSpecsActive` method is a well-structured piece of code that effectively employs caching and override mechanisms. To improve its quality, focus on defining all variables, handling potential null references, simplifying conditional logic, and implementing consistent naming conventions. Additionally, incorporating comprehensive error handling will ensure that the code gracefully manages unexpected scenarios, enhancing its reliability and maintainability. By implementing these recommendations, the code will be more robust, maintainable, and easier to understand.

## Original Code

```c#
public Dictionary<string, string> CheckSpecsActive(string testKind, string modelNumber, Dictionary<string, string> OverrideDict)     {         Dictionary<string, string> dictSpecs = new Dictionary<string, string>();         ConfigObject ConfigObj = new ConfigObject(path);         Specs = new SpecsManager(ConfigObj.GetConfigObject(ConfigConst.SPECS_CONFIG), false);         string key = testKind + "_" + modelNumber;         //Check cache specs first         if (SpecCache.TryGet(key, out var CacheSpecs))         {             return CacheSpecs;         }         else         {             Specs.SetActiveSpecs(testKind, modelNumber);             //Store the result in CacheSpecs             SpecCache.Set(key, CacheSpecs);         }           if (OverrideDict.Count > 0) // if Specs is null or if specs was not null but Specs was Overridden still check         {             foreach (var pair in OverrideDict)             {                 string specname = pair.Key;                 string specvalue = pair.Value;                 // Use specname and specvalue here                 Specs.OverrideSpec(specname, specvalue);             }             dictSpecs = Specs.GetSpecsAsDictionary(testKind, modelNumber);          }         else if (OverrideDict.Count == 0 && Specs == null)         {             dictSpecs = Specs.GetSpecsAsDictionary(testKind, modelNumber);         }         else if (Specs != null)         {             //do nothing we don't need to go and ask for Specs again         }           return dictSpecs;     }
```
